> ### Note on Labs and Assignments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 6: Data Cleaning

## Outline

- Load and inspect a new dataset (Megatelco)
- Fix column names and data types
- Handle missing values
- Remove duplicate rows
- Review and remove outliers
- Reflect on data quality

In this lab, we’ll clean the data to get it ready for transformations and analysis.

We will continue working with this dataset in **Lab 7**, where we will create new features and apply transformations.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do you use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [3]:
# create a copy of your dataset for use in part 4
copied_df = df.copy(deep=True)

## Part 1: Review Column Names and Structure

Before cleaning, check the structure of the dataset:

- Are column names consistent (lowercase, no spaces)?
- Are there any typos or redundant labels?
- Do the rows and columns appear aligned? (Are all the columns the same size? Are all the rows the same size?)

Why this matters:
Inconsistent or messy column names can break code and make analysis harder to follow.




In [4]:
# Standardize column names: lowercase, no spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

## Part 2: Convert Data Types

Before analysis, make sure each column is stored in the correct format. This helps avoid calculation errors, makes plotting smoother, and ensures models interpret the data correctly.

Think about:
- Are numbers accidentally stored as strings?
- Should repeated text values be converted to categories?
- Are "yes"/"no" columns better represented as binary (0/1) or categorical types?

Fixing data types now saves time and avoids issues later in your workflow.




In [5]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert yes/no to binary categories
df['considering_change_of_plan'] = df['considering_change_of_plan'].map({'yes': 1, 'no': 0}).astype('category')

# Convert categorical text columns
df['college'] = df['college'].astype('category')
df['house'] = df['house'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['operating_system'] = df['operating_system'].astype('category')

# Convert numeric-looking columns from object to float
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['data_overage_mb'] = pd.to_numeric(df['data_overage_mb'], errors='coerce')
df['data_leftover_mb'] = pd.to_numeric(df['data_leftover_mb'], errors='coerce')
df['data_mb_used'] = pd.to_numeric(df['data_mb_used'], errors='coerce')
df['text_message_count'] = pd.to_numeric(df['text_message_count'], errors='coerce')
df['over_15mins_calls_per_month'] = pd.to_numeric(df['over_15mins_calls_per_month'], errors='coerce')
df['average_call_duration'] = pd.to_numeric(df['average_call_duration'], errors='coerce')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                          category
handset_price                   float64
over_1

### 🔧 Try It Yourself – Part 2

1. Convert the `leave` column from "yes"/"no" to binary (`1`/`0`) and make it a **category**
2. Convert `reported_usage_level` to a **categorical** type
3. Convert `house` to an **integer** type
3. Use `.info()` to confirm the changes


In [6]:
# Convert 'leave' column to binary (1/0) and category type
df['leave'] = df['leave'].map({'LEAVE': 1, 'STAY': 0}).astype('category')

# Convert 'reported_usage_level' to a categorical type
df['reported_usage_level'] = df['reported_usage_level'].astype('category')

# Convert 'house' to an integer type
df['house'] = df['house'].astype('int64')

# Use .info() to confirm the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      15016 non-null  category
 1   income                       15006 non-null  float64 
 2   data_overage_mb              15016 non-null  int64   
 3   data_leftover_mb             14916 non-null  float64 
 4   data_mb_used                 14916 non-null  float64 
 5   text_message_count           15016 non-null  int64   
 6   house                        15016 non-null  int64   
 7   handset_price                14916 non-null  float64 
 8   over_15mins_calls_per_month  15013 non-null  float64 
 9   average_call_duration        14916 non-null  float64 
 10  reported_satisfaction        15016 non-null  category
 11  reported_usage_level         15016 non-null  category
 12  considering_change_of_plan   14201 non-null  category
 13  l

## Part 3: Handle Missing Values

Missing data can break charts, skew stats, and disrupt models — so it needs to be handled carefully.

### Think about:
- Are the missing values random or patterned?
- Can we drop rows, or do we need to fill them?
- Should we use mean, median, or something else?

### Guidelines:
- Drop rows if there are only a few missing and the columns associated with them are essential to keep intact
- Use median to replace outliers in numeric columns
- Use 0 if the missing value means “none” (e.g. If the value was in response to: “do you have a history of chronic illness?” and the value was just left blank, we can assume that that blank just means “none” (the patient has no history of chronic illness))
- Use mode to replace categorical values

Cleaning missing values early avoids bigger problems later.

-----


**Note on `.loc` and Warnings** - When assigning values to a DataFrame, especially after filtering or copying, it's best to use `.loc` to avoid **`SettingWithCopyWarning`**. This ensures that you're updating the original data and not a temporary view of it.


In [7]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing 'income' (if very few)
df = df.dropna(subset=['income']).copy()

# Fill missing 'data_leftover_mb' with 0 if it logically means no leftover data
df.loc[:, 'data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill 'average_call_duration' with median if necessary
df.loc[:, 'average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())

# Fill 'data_mb_used' with median
df.loc[:, 'data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Confirm updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 college                          0
income                          10
data_overage_mb                  0
data_leftover_mb               100
data_mb_used                   100
text_message_count               0
house                            0
handset_price                  100
over_15mins_calls_per_month      3
average_call_duration          100
reported_satisfaction            0
reported_usage_level             0
considering_change_of_plan     815
leave                            0
id                               0
operating_system                 0
dtype: int64

Missing values after handling:
 college                          0
income                           0
data_overage_mb                  0
data_leftover_mb                 0
data_mb_used                     0
text_message_count               0
house                            0
handset_price                    0
over_15mins_calls_per_month      3
average_call_duration            0
reported_satisfa

### 🔧 Try It Yourself – Part 3


There are still some missing values in:

- `over_15mins_calls_per_month`
- `considering_change_of_plan`

Decide how to handle them based on what makes the most sense:

- Should you fill them with 0, the median, or something else?
- For categories, would a placeholder like "unknown" or the most common value work?
- Or is it better to drop those rows?

1. Write and execute code to handle the missing values in the remaining two columns.
2. Use `df.isnull().sum()` to confirm all missing values are handled.



In [9]:
df['over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].fillna(0)
df['considering_change_of_plan'] = df['considering_change_of_plan'].fillna(0)
df.isnull().sum()

,0
college,0
income,0
data_overage_mb,0
data_leftover_mb,0
data_mb_used,0
text_message_count,0
house,0
handset_price,0
over_15mins_calls_per_month,0
average_call_duration,0


## Part 4: Remove Duplicate Rows

Sometimes the same row appears more than once due to data entry or processing mistakes. It's important to check for and remove these duplicates.

Think about:
- Are there rows that are exactly the same?
- If duplicates exist, should you keep the first one, the last one, or none?

Why this matters:
Duplicate rows can inflate totals, distort statistics, and lead to inaccurate conclusions.


In [10]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove them, keeping the first occurrence
df = df.drop_duplicates()

# Confirm result
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14989


### 🔧 Try It Yourself – Part 4

1. Use `copied_df.duplicated().sum()` to count how many duplicates are in your dataset.
2. Try using `copied_df.drop_duplicates(keep='last')` instead — what changes?
3. Explore whether duplicate rows share the same ID or just values across all columns and comment on your observation.


In [20]:
print(f"Number of duplicate rows in copied_df: {copied_df.duplicated().sum()}")
deduped_df = copied_df.drop_duplicates(keep='last')
print(f"Remaining rows after dropping duplicates: {len(deduped_df)}")
duplicate_rows = copied_df[copied_df.duplicated(keep=False)]
print("Duplicate rows:")
display(duplicate_rows.sort_values(by='id'))

Number of duplicate rows in copied_df: 17
Remaining rows after dropping duplicates: 14999
Duplicate rows:


,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
69,zero,-65000.0,0,0.0,6444.0,160,542857,993.0,0.0,11.0,low,high,no,STAY,467,IOS
75,zero,-65000.0,0,0.0,6444.0,160,542857,993.0,0.0,11.0,low,high,no,STAY,467,IOS
49,zero,400865.0,159,0.0,5656.0,130,771829,1292.0,22.0,13.0,low,high,yes,STAY,705,IOS
56,zero,400865.0,159,0.0,5656.0,130,771829,1292.0,22.0,13.0,low,high,yes,STAY,705,IOS
66,one,414896.0,219,84.0,5008.0,56,663443,938.0,21.0,1.0,low,high,yes,LEAVE,1204,IOS
72,one,414896.0,219,84.0,5008.0,56,663443,938.0,21.0,1.0,low,high,yes,LEAVE,1204,IOS
64,zero,195320.0,0,75.0,3791.0,186,1012042,617.0,0.0,2.0,low,low,yes,LEAVE,3636,Android
63,zero,195320.0,0,75.0,3791.0,186,1012042,617.0,0.0,2.0,low,low,yes,LEAVE,3636,Android
67,one,291877.0,56,13.0,6319.0,68,1354850,382.0,3.0,5.0,low,low,no,STAY,4743,Android
73,one,291877.0,56,13.0,6319.0,68,1354850,382.0,3.0,5.0,low,low,no,STAY,4743,Android


🔧 Add comment here:they are true record duplicates with the same ID. There are duplicate records of the same customer, not different customers with coincidentally identical attributes.

## Part 5: Identify and Remove Obvious Outliers

Outliers are values that fall far outside the normal range. They can come from data entry mistakes or rare cases.

- Use summary statistics or visual tools (like boxplots) to find them.
- Look for clearly unrealistic values — e.g., negative prices or extremely high data usage.
- Decide how to handle them:
  - Remove if they’re errors.
  - Keep if they’re valid but rare — or cap them if needed.

Outliers can distort averages, stretch visualizations, and mislead models, so it’s important to address them carefully.



In [21]:
# Remove negative or nonsensical values using business rules

# Example: remove rows where 'handset_price' is negative
df = df[df['handset_price'] >= 0]

# Example: remove rows with unusually long call durations
df = df[df['average_call_duration'] < 1000]

# Example: remove rows with extremely high text message counts
df = df[df['text_message_count'] < 1000]

# View shape after outlier filtering
print("Shape after removing obvious outliers:", df.shape)


Shape after removing obvious outliers: (14986, 16)


### 🔧 Try It Yourself – Part 5

1. Use `df.describe()` to look for columns with extreme minimum or maximum values.
2. Set a threshold for what you think is "too high" or "too low" for:
  - `data_mb_used`
  - `over_15mins_calls_per_month`
  - `income`
3. Remove those outliers using boolean filtering like `df = df[df['column'] < threshold]`

In [22]:
# Use df.describe() to look for columns with extreme minimum or maximum values.
print("Summary statistics before removing outliers:")
display(df.describe())

# Set a threshold for what you think is "too high" or "too low" for:
# data_mb_used, over_15mins_calls_per_month, and income

# Based on the describe output, let's set thresholds. These are examples and can be adjusted.
# For data_mb_used, the max is significantly higher than the 75th percentile. Let's use the 99th percentile as a threshold.
data_mb_used_threshold = df['data_mb_used'].quantile(0.99)

# For over_15mins_calls_per_month, the max is also quite high. Let's use the 99th percentile.
over_15mins_calls_threshold = df['over_15mins_calls_per_month'].quantile(0.99)

# For income, there might be some very high values. Let's use the 99th percentile.
income_threshold = df['income'].quantile(0.99)

print(f"\nSuggested threshold for data_mb_used: {data_mb_used_threshold:.2f}")
print(f"Suggested threshold for over_15mins_calls_per_month: {over_15mins_calls_threshold:.2f}")
print(f"Suggested threshold for income: {income_threshold:.2f}")

# Remove those outliers using boolean filtering
df = df[df['data_mb_used'] < data_mb_used_threshold]
df = df[df['over_15mins_calls_per_month'] < over_15mins_calls_threshold]
df = df[df['income'] < income_threshold]


# After filtering, confirm the changes by running df.describe() again
print("\nSummary statistics after removing outliers:")
display(df.describe())

Summary statistics before removing outliers:


,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,14986.000000,14986.000000,14986.000000,14986.000000,14986.000000,1.498600e+04,14986.000000,14986.000000,14986.000000,14986.000000
mean,241977.965768,153.510543,37.262045,4201.313760,135.626985,8.771324e+05,795.018617,10.567530,9.731149,11861.289604
std,109612.549926,113.011748,28.120274,2196.751708,48.835016,2.870422e+05,1235.982599,8.402473,5.178082,6812.499283
min,-65000.000000,0.000000,0.000000,400.000000,52.000000,-4.630000e+02,215.000000,0.000000,1.000000,2.000000
25%,147808.250000,54.000000,12.000000,2304.000000,93.000000,6.442570e+05,499.250000,3.000000,5.000000,6139.000000
50%,241653.000000,151.000000,34.000000,4221.000000,135.000000,8.762530e+05,777.000000,9.000000,10.000000,11763.500000
75%,336442.000000,242.000000,62.000000,6063.000000,178.000000,1.098814e+06,1062.000000,17.000000,14.000000,17398.000000
max,432000.000000,380.000000,89.000000,8000.000000,220.000000,1.456389e+06,125000.000000,35.000000,19.000000,25354.000000



Suggested threshold for data_mb_used: 7929.15
Suggested threshold for over_15mins_calls_per_month: 33.00
Suggested threshold for income: 427995.95

Summary statistics after removing outliers:


,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,14536.000000,14536.000000,14536.000000,14536.000000,14536.000000,1.453600e+04,14536.000000,14536.000000,14536.000000,14536.000000
mean,239999.645982,153.542309,37.137658,4158.050977,135.665451,8.768554e+05,795.041414,10.325812,9.731081,11848.914213
std,108536.120981,113.051336,28.108397,2175.423064,48.799963,2.869909e+05,1253.666068,8.099421,5.181135,6822.176872
min,-65000.000000,0.000000,0.000000,400.000000,52.000000,-4.630000e+02,215.000000,0.000000,1.000000,2.000000
25%,146229.000000,54.000000,12.000000,2275.000000,93.000000,6.435070e+05,499.000000,3.000000,5.000000,6101.500000
50%,239693.500000,151.000000,34.000000,4205.000000,135.000000,8.765105e+05,777.000000,9.000000,10.000000,11749.500000
75%,334030.250000,242.000000,62.000000,6014.000000,178.000000,1.098404e+06,1061.000000,17.000000,14.000000,17389.250000
max,427991.000000,380.000000,89.000000,7929.000000,220.000000,1.456389e+06,125000.000000,32.000000,19.000000,25354.000000


## Part 6: Handle Outliers Using Quantiles

Instead of removing outliers, we can limit their impact by capping extreme values — a method known as **Winsorizing**.

### How to Do It:
- Use `.quantile()` to identify the 1st and 99th percentiles (or other thresholds).
- Use `.clip()` to cap values within that range.

This keeps your dataset intact while reducing the influence of extreme values on your analysis or model.



In [23]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)



### 🔧 Try It Yourself – Part 6

1. Use `.quantile([0.01, 0.99])` to find the range for:
  - `text_message_count`
  - `over_15mins_calls_per_month`
2. Apply `.clip(lower=..., upper=...)` to reduce the impact of those outliers
3. Compare the `.describe()` output before and after clipping and comment on what you observe


In [25]:
text_min, text_max = df['text_message_count'].quantile([0.01, 0.99])
calls_min, calls_max = df['over_15mins_calls_per_month'].quantile([0.01, 0.99])

print(f"1st and 99th percentiles for text_message_count: ({text_min:.2f}, {text_max:.2f})")
print(f"1st and 99th percentiles for over_15mins_calls_per_month: ({calls_min:.2f}, {calls_max:.2f})")

print("\nThese percentiles represent the boundaries within which 98% of the data falls. Values outside this range are considered potential outliers.")

print("\nSummary statistics before clipping:")
display(df[['text_message_count', 'over_15mins_calls_per_month']].describe())

df.loc[:, 'text_message_count'] = df['text_message_count'].clip(lower=text_min, upper=text_max)
df.loc[:, 'over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].clip(lower=calls_min, upper=calls_max)

print("\nSummary statistics after clipping:")
display(df[['text_message_count', 'over_15mins_calls_per_month']].describe())

1st and 99th percentiles for text_message_count: (53.00, 219.00)
1st and 99th percentiles for over_15mins_calls_per_month: (0.00, 29.00)

These percentiles represent the boundaries within which 98% of the data falls. Values outside this range are considered potential outliers.

Summary statistics before clipping:


,text_message_count,over_15mins_calls_per_month
count,14536.000000,14536.000000
mean,135.664695,10.305793
std,48.776055,8.049997
min,53.000000,0.000000
25%,93.000000,3.000000
50%,135.000000,9.000000
75%,178.000000,17.000000
max,219.000000,29.000000



Summary statistics after clipping:


,text_message_count,over_15mins_calls_per_month
count,14536.000000,14536.000000
mean,135.664695,10.305793
std,48.776055,8.049997
min,53.000000,0.000000
25%,93.000000,3.000000
50%,135.000000,9.000000
75%,178.000000,17.000000
max,219.000000,29.000000


🔧 Add comment here: The summary statistics for text_message_count and over_15mins_calls_per_month remained unchanged before and after clipping, indicating no significant shift in their distributions. Both variables maintained identical values for the minimum, maximum, mean, standard deviation, and quartiles: text_message_count ranged from 53 to 219, while over_15mins_calls_per_month spanned from 0 to 29. This suggests that all data points were already within the thresholds of the 1st and 99th percentiles; therefore, the clipping process neither removed nor altered any values. Essentially, there were no extreme outliers beyond these cutoffs in either column—demonstrating that this part of the dataset is relatively clean as it stands.

## 🔧 Part 7: Reflection (100 words or less per question)

1. Which step fixed the most issues in the dataset?
2. What surprised you about the structure or values?
3. Do you feel this data is now ready for transformation in Lab 7?


1. The step that fixed the most issues was handling missing values, particularly in the considering_change_of_plan and over_15mins_calls_per_month columns. Addressing these gaps ensured consistency and prevented potential bias in later analysis. This step resolved the majority of data integrity problems compared to others.

2. I was surprised to find duplicate rows with the same ID and identical values across all columns. This suggested redundancy in the dataset that could have inflated certain patterns if left untreated.

3. Yes, the dataset is now much cleaner missing values, duplicates, and outliers addressed. The categorical conversions and type adjustments also prepare it for transformation. While there may still be small anomalies, it is in a strong position to move forward for modeling and deeper analysis in Lab 7.

## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [26]:
!jupyter nbconvert --to html "lab_06_LastnameFirstname.ipynb"

[NbConvertApp] Converting notebook lab_06_LastnameFirstname.ipynb to html
[NbConvertApp] Writing 397772 bytes to lab_06_LastnameFirstname.html
